# Video Activity Classifier Workshop

In this workshop, we will be training a classifier using body positions extracted from video.

![photo](assets/istockphoto-476741742.jpg)

This follows up from [04_pose_estimation.ipynb](04_pose_estimation.ipynb).

In [5]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import sys
import numpy as np
from sklearn.metrics.pairwise import paired_distances

# requires: conda install opencv
import cv2

plt.style.use('seaborn-white')

In [6]:
def keypoints_to_dataframe(keypoints):
    """Converts a flat keypoints list (x1, y1, c1, x2, y2, c2) into a pandas DataFrame"""
    return pd.DataFrame({'x': keypoints[::3], 'y': keypoints[1::3], 'c': keypoints[2::3]})

def get_centroid(coordinates, threshold=0.1):
    """Computes the centroid of a given 2 dimensional vector"""
    x = coordinates[coordinates.c > threshold].x
    y = coordinates[coordinates.c > threshold].y
    
    return [sum(x)/len(x), sum(y)/len(y)]

def get_centroids(frame):
    """Returns the centroid for each person as a list of (x, y) coordinates"""
    return np.array([get_centroid(keypoints_to_dataframe(person['pose_keypoints_2d'])) for person in frame['people']])

def get_closest_index(centroid, other_frame):
    """Find closest index in other_frame from a given centroid"""
    other_centroids = get_centroids(other_frame)
    return np.argmin(paired_distances(np.ones(other_centroids.shape) * centroid, other_centroids))

## Extract keypoints from the videos using OpenPose

1. Modify the paths under `HSS_DIR` and `OPENPOSE_DIR`.

2. Run the code below which calls OpenPoseDemo to extract the keypoints from the data videos.

In [13]:
import glob
import os

############# CHANGE BELOW PATH ################
HSS_DIR=r'D:\S-HSS\Workshop\hss'
############# CHANGE ABOVE PATH ################

############# CHANGE BELOW PATH ################
OPENPOSE_DIR=r'D:\S-HSS\Workshop\openpose-1.5.1-binaries-win64-only_cpu-python-flir-3d\openpose'
############# CHANGE ABOVE PATH ################

classes = ['awake', 'sleep']

cwd = os.getcwd()
os.chdir(OPENPOSE_DIR)
for c in classes:
    data_path=os.path.join(HSS_DIR, r'data\dozing', c)
    files = glob.glob(os.path.join(data_path, '*.avi'))
    
    for f in files:
        name, _ = f.split('.')
        input_path=os.path.join(data_path, f)
        output_path=os.path.join(data_path, name)

        cmd = f'bin\\OpenPoseDemo.exe --part_candidates --write_json {output_path} --video {input_path} \
        --frame_first 20 --frame_step 1 --frame_last 50'
        print(cmd)
        os.system(cmd)

os.chdir(cwd)

bin\OpenPoseDemo.exe --part_candidates --write_json D:\S-HSS\Workshop\hss\data\dozing\awake\1 --video D:\S-HSS\Workshop\hss\data\dozing\awake\1.avi         --frame_first 20 --frame_step 1 --frame_last 50
bin\OpenPoseDemo.exe --part_candidates --write_json D:\S-HSS\Workshop\hss\data\dozing\awake\2 --video D:\S-HSS\Workshop\hss\data\dozing\awake\2.avi         --frame_first 20 --frame_step 1 --frame_last 50
bin\OpenPoseDemo.exe --part_candidates --write_json D:\S-HSS\Workshop\hss\data\dozing\awake\3 --video D:\S-HSS\Workshop\hss\data\dozing\awake\3.avi         --frame_first 20 --frame_step 1 --frame_last 50
bin\OpenPoseDemo.exe --part_candidates --write_json D:\S-HSS\Workshop\hss\data\dozing\sleep\1 --video D:\S-HSS\Workshop\hss\data\dozing\sleep\1.avi         --frame_first 20 --frame_step 1 --frame_last 50
bin\OpenPoseDemo.exe --part_candidates --write_json D:\S-HSS\Workshop\hss\data\dozing\sleep\2 --video D:\S-HSS\Workshop\hss\data\dozing\sleep\2.avi         --frame_first 20 --frame_ste